<a href="https://www.kaggle.com/code/sscarecrow/phowhisperforaic?scriptVersionId=258914893" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Libraries Install

In [1]:
!pip -q install "transformers>=4.41" "accelerate>=0.30" torch torchaudio soundfile librosa
!ffmpeg -version >/dev/null 2>&1 || (apt-get -y update && apt-get -y install ffmpeg)
!pip install -q transformers sentence-transformers torch accelerate einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.2 MB/s eta 0:00:00:00:0100:01


# Libraries and Helper Functions

In [2]:
import os, json, math, subprocess
from pathlib import Path
from datetime import timedelta
from typing import List, Dict, Any
import torch
import soundfile as sf
import librosa
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import numpy as np
import json

INPUT_DIR  = "/kaggle/working/videos"        
OUTPUT_DIR = "/kaggle/working/transcripts"   
MODEL_ID   = "vinai/PhoWhisper-large"
LANGUAGE   = "vi"
AUDIO_SR = 16000
AUDIO_CH = 1
VIDEO_EXTS = {".mp4", ".mkv", ".mov", ".avi", ".flv", ".webm"}
AUDIO_EXTS = {".wav", ".mp3", ".m4a", ".aac", ".flac", ".ogg", ".wma"}
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
def run_cmd(cmd):
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if p.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(cmd)}\nSTDERR:\n{p.stderr}")
def has_stream_audio(input_path: str) -> bool:
    try:
        p = subprocess.run(
            ["ffprobe","-v","error","-select_streams","a",
             "-show_entries","stream=index","-of","csv=p=0", input_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        return bool(p.stdout.strip())
    except Exception:
        return True
def extract_wav_16k_mono(input_media: str, out_wav: str) -> None:
    cmd = [
        "ffmpeg","-y","-i", input_media,
        "-vn","-acodec","pcm_s16le",
        "-ar", str(AUDIO_SR), "-ac", str(AUDIO_CH),
        out_wav
    ]
    run_cmd(cmd)
def list_media_files(folder: str):
    p = Path(folder)
    files = []
    for ext in list(VIDEO_EXTS | AUDIO_EXTS):
        files.extend(p.rglob(f"*{ext}"))
    return sorted(files)
def format_timestamp(seconds: float) -> str:
    total_ms = int(seconds * 1000)
    h = total_ms // 3_600_000
    m = (total_ms % 3_600_000) // 60_000
    s = (total_ms % 60_000) // 1000
    ms = total_ms % 1000
    return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"
def write_txt(path: str, full_text: str):
    with open(path, "w", encoding="utf-8") as f:
        f.write(full_text.strip() + "\n")
def write_json(path: str, segments: List[Dict[str,Any]], meta: Dict[str,Any]):
    payload = {
        "meta": meta,
        "segments": segments,
        "text": " ".join(seg["text"] for seg in segments).strip()
    }
    with open(path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
def write_srt(path: str, segments: List[Dict[str,Any]]):
    lines = []
    for i, seg in enumerate(segments, 1):
        lines += [str(i),
                  f"{format_timestamp(seg['start'])} --> {format_timestamp(seg['end'])}",
                  seg["text"].strip(),
                  ""]
    with open(path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

print("Config loaded.")

2025-08-27 17:42:48.613658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756316568.807763      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756316568.863558      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Config loaded.


# Model (PhoWhisper)

In [3]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, pipeline
device = 0 if torch.cuda.is_available() else -1
dtype  = torch.float16 if torch.cuda.is_available() else torch.float32
processor = AutoProcessor.from_pretrained(MODEL_ID)
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    MODEL_ID,
    torch_dtype=dtype,
    low_cpu_mem_usage=True
)
gen = model.generation_config
gen.forced_decoder_ids = processor.get_decoder_prompt_ids(language="vi", task="transcribe")
gen.language = "vi"
gen.task = "transcribe"
model.generation_config = gen
if device == 0:
    model = model.to("cuda")
asr = pipeline(
    task="automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    device=device
)
print("PhoWhisper loaded:", MODEL_ID)

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


PhoWhisper loaded: vinai/PhoWhisper-large


# Extracting a file

In [ ]:
SINGLE_INPUT = "/kaggle/input/aic-small-2024/Videos_L22_a/video/L22_V002.mp4" 
CUSTOM_NAME  = None                                  
src = Path(SINGLE_INPUT)
assert src.exists(), f"Không tìm thấy file: {src}"
stem = CUSTOM_NAME if CUSTOM_NAME else src.stem
out_base = Path(OUTPUT_DIR) / stem
out_base.parent.mkdir(parents=True, exist_ok=True)
wav_path  = str(out_base.with_suffix(".wav"))
txt_path  = str(out_base.with_suffix(".txt"))
json_path = str(out_base.with_suffix(".json"))
srt_path  = str(out_base.with_suffix(".srt"))
print(f"=== Xử lý 1 file: {src} ===")
if src.suffix.lower() in VIDEO_EXTS and not has_stream_audio(str(src)):
    raise SystemExit("File không có audio stream. Không thể tạo transcript.")
extract_wav_16k_mono(str(src), wav_path)
print("-> Đã tách WAV 16k mono.")
audio, sr = librosa.load(wav_path, sr=AUDIO_SR, mono=True)
result = asr(
    audio,                
    return_timestamps="phrase"  
)
full_text = result["text"].strip()
segments_out = []
chunks = result.get("chunks", None)
if chunks:
    for i, ck in enumerate(chunks):
        start = float(ck["timestamp"][0]) if ck["timestamp"][0] is not None else 0.0
        end   = float(ck["timestamp"][1]) if ck["timestamp"][1] is not None else max(start, 0.0)
        segments_out.append({
            "id": i,
            "start": start,
            "end": end,
            "text": ck["text"].strip()
        })
else:
    duration = librosa.get_duration(y=audio, sr=sr)
    segments_out.append({"id": 0, "start": 0.0, "end": float(duration), "text": full_text})
meta = {
    "source": str(src),
    "wav_16k_mono": wav_path,
    "language": LANGUAGE,
    "model": MODEL_ID,
    "duration": float(librosa.get_duration(y=audio, sr=sr))
}
write_txt(txt_path, full_text)
write_json(json_path, segments_out, meta)
write_srt(srt_path, segments_out)
print(f"-> OK: {txt_path}")
print(f"-> OK: {json_path}")
print(f"-> OK: {srt_path}")


=== Xử lý 1 file: /kaggle/input/aic-small-2024/Videos_L22_a/video/L22_V002.mp4 ===
-> Đã tách WAV 16k mono.


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [(1, 50278), (2, 50359), (3, 50363)] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.


# Extracting all files in a folder

In [ ]:
# media_files = list_media_files(INPUT_DIR)
# if not media_files:
#     raise SystemExit(f"Không tìm thấy video/audio trong: {INPUT_DIR}")
# print(f"Đã tìm thấy {len(media_files)} tệp.")
# for src in media_files:
#     src = Path(src)
#     stem = src.stem
#     out_base = Path(OUTPUT_DIR) / stem
#     out_base.parent.mkdir(parents=True, exist_ok=True)
#     wav_path  = str(out_base.with_suffix(".wav"))
#     txt_path  = str(out_base.with_suffix(".txt"))
#     json_path = str(out_base.with_suffix(".json"))
#     srt_path  = str(out_base.with_suffix(".srt"))
#     print(f"\n=== Xử lý: {src} ===")
#     if src.suffix.lower() in VIDEO_EXTS and not has_stream_audio(str(src)):
#         print("-> Bỏ qua: file không có audio stream.")
#         continue
#     extract_wav_16k_mono(str(src), wav_path)
#     print("-> Đã tách WAV 16k mono.")
#     audio, sr = librosa.load(wav_path, sr=AUDIO_SR, mono=True)
#     result = asr(
#         audio,
#         generate_kwargs={"language": LANGUAGE, "task": "transcribe"},
#         return_timestamps="phrase"
#     )
#     full_text = result["text"].strip()
#     segments_out = []
#     chunks = result.get("chunks", None)
#     if chunks:
#         for i, ck in enumerate(chunks):
#             start = float(ck["timestamp"][0]) if ck["timestamp"][0] is not None else 0.0
#             end   = float(ck["timestamp"][1]) if ck["timestamp"][1] is not None else max(start, 0.0)
#             segments_out.append({
#                 "id": i,
#                 "start": start,
#                 "end": end,
#                 "text": ck["text"].strip()
#             })
#     else:
#         duration = librosa.get_duration(y=audio, sr=sr)
#         segments_out.append({"id": 0, "start": 0.0, "end": float(duration), "text": full_text})

#     meta = {
#         "source": str(src),
#         "wav_16k_mono": wav_path,
#         "language": LANGUAGE,
#         "model": MODEL_ID,
#         "duration": float(librosa.get_duration(y=audio, sr=sr))
#     }
#     write_txt(txt_path, full_text)
#     write_json(json_path, segments_out, meta)
#     write_srt(srt_path, segments_out)

#     print(f"-> OK: {txt_path}")
#     print(f"-> OK: {json_path}")
#     print(f"-> OK: {srt_path}")
